In [25]:
from process_jobs import load_api_key, get_openai_embedding_function, load_and_clean_data, get_chroma_collection, build_vector_base, configure_retrievers, run_sample_query
import pandas as pd
import chromadb

In [26]:
# --- Configuration ---
JOB_DATA_FILE = 'jobsspider_2024-01-23T04-34-07+00-00.csv'
CHROMA_HOST = 'localhost'
CHROMA_PORT = 8000
COLLECTION_NAME = "job"

print("Démarrage du script de traitement des offres d'emploi...")

# 1. Charger la clé API et la fonction d'embedding
api_key = load_api_key()
if not api_key:
    exit(1) # Arrêter si la clé API n'est pas trouvée
    
openai_embedding_func = get_openai_embedding_function(api_key)
if not openai_embedding_func:
        exit(1) # Arrêter si la fonction d'embedding échoue

# 2. Charger et nettoyer les données
job_data = load_and_clean_data(JOB_DATA_FILE)
if job_data is None:
    exit(1) # Arrêter si les données ne peuvent pas être chargées/nettoyées

# 3. Initialiser le client ChromaDB
try:
    chroma_client = chromadb.HttpClient(host=CHROMA_HOST, port=CHROMA_PORT)
    # Essayer de vérifier la connexion (ex: lister les collections)
    chroma_client.list_collections() 
    print(f"Client ChromaDB connecté à {CHROMA_HOST}:{CHROMA_PORT}.")
except Exception as e:
    print(f"Erreur de connexion au serveur ChromaDB à {CHROMA_HOST}:{CHROMA_PORT}: {e}")
    print("Vérifiez que le serveur ChromaDB est lancé (par exemple avec `chroma run --path /path/to/db`)")
    exit(1)

# 4. Obtenir/Créer la collection ChromaDB
job_collection = get_chroma_collection(chroma_client, COLLECTION_NAME, openai_embedding_func)
if job_collection is None:
    exit(1) # Arrêter si la collection ne peut pas être obtenue/créée

# 5. Construire/Mettre à jour la base vectorielle (optionnel si déjà fait)
# Décommentez la ligne suivante pour exécuter l'ajout à chaque fois
# build_vector_base(job_data, job_collection) 
# OU: Vérifier si la collection est vide avant de construire
if job_collection.count() < len(job_data): # Simple vérification, peut être affinée
        print("\nLa collection semble incomplète ou vide, lancement de build_vector_base...")
        build_vector_base(job_data, job_collection)
else:
        print(f"\nLa collection '{COLLECTION_NAME}' contient déjà {job_collection.count()} éléments. Saut de build_vector_base.")


# 6. Configurer les Retrievers
hybrid_retriever = configure_retrievers(job_data, chroma_client, COLLECTION_NAME, openai_embedding_func)

# 7. Exécuter une requête exemple
if hybrid_retriever:
    SAMPLE_QUERY = "Data scientist avec experience en Python et machine learning à Paris"
    run_sample_query(hybrid_retriever, SAMPLE_QUERY)
else:
    print("\nLe retriever hybride n'a pas pu être configuré. Le test de requête est sauté.")
    
print("\nScript terminé.") 

Démarrage du script de traitement des offres d'emploi...
Fonction d'embedding OpenAI prête.
Données chargées depuis jobsspider_2024-01-23T04-34-07+00-00.csv. Shape: (934, 9)
Nettoyage des descriptions de poste...
Nettoyage terminé.

Exemple après nettoyage (avec localisation) :
frg consulting recrutement pour l'un de ses clients, un cabinet de conseil à taille humaine basé à aix-en-provence, façonnant l'avenir des solutions innovantes pour leurs clients. dans le cadre de leur expansion, ils recherchent un data analyst junior h/f enthousiaste, spécialisé dans power bi, pour rejoindre une équipe dynamique. responsabilités analyser et extraire des insights significatifs à partir de données complexes. concevoir, développer et entretenir des tableaux de bord interactifs avec power bi. collaborer étroitement avec les équipes internes pour comprendre les exigences en données et proposer des solutions adaptées. contribuer à la collecte, au nettoyage et à la gestion efficace des données. profil

c:\Users\Yvank\OneDrive - ESIGELEC\Bureau\chat_cv\chat-cv\process_jobs.py:190: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  hybrid_results = retriever.get_relevant_documents(query)
